# Left Bundle Branch Block

In [162]:
import os
import pdb

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
import wfdb

from bc.beats import get_beats, get_beat_bank
from bc.io import ann_to_df

%matplotlib qt

base_dir = os.path.abspath('..')
data_dir = os.path.join(base_dir, 'data')

# Table of record names and the beat types they contain
beat_table = pd.read_csv(os.path.join(data_dir, 'beat-types.csv'), dtype={'record':object})
beat_table.set_index('record', inplace=True)

In [ ]:
# Normal beats
n_beats, n_centers = get_beat_bank(data_dir=data_dir, beat_table=beat_table, wanted_type='N')
for i in range(5):
    plt.figure()
    plt.plot(n_beats[i])
    plt.plot(n_centers[i], n_beats[i][n_centers[i]], 'r*')
    plt.show()

In [9]:
# Left bundle branch block beats
l_beats, l_centers = get_beat_bank(data_dir=data_dir, beat_table=beat_table, wanted_type='L')
for i in range(5):
    plt.figure()
    plt.plot(l_beats[i])
    plt.plot(l_centers[i], l_beats[i][l_centers[i]], 'r*')
    plt.show()

In [15]:
for i in range(1):
    plt.figure()
    plt.plot(n_beats[i])
    plt.plot(n_centers[i], n_beats[i][n_centers[i]], 'r*')
    plt.show()

In [177]:
sig, center = n_beats[0], centers[0]
# sig_smooth = signal.filtfilt(b=[1] * 10, a=[1], x=sig) / 40
analytic_signal = signal.hilbert(sig)

amplitude_envelope = np.abs(analytic_signal)

def segment_beat(sig, qrs_ind, fs=360, max_qrs_duration=0.2):
    """
    Segment a single heartbeat, given the signal and its qrs index
    """
    # Moving average samples
    
    
    ma_samps = 10
    sig = signal.filtfilt(b=[1] * ma_samps, a=[1], x=sig)
    
    # Isolate the qrs complex
    max_qrs_width = int(max_qrs_duration * fs)
    h_max_qrs_width = max_qrs_width // 2
    
    dy = np.diff(sig)
    abs_dy = dy ** 2
    
    print(max_qrs_width, h_max_qrs_width)
    # Crawl left and right until the moving average gradient remains relatively flat
    # Left
    for i in range(qrs_ind, qrs_ind-h_max_qrs_width, -1):
        ma_gradient_l = np.mean(abs_dy[i-ma_samps:i])
        ma_gradient_r = np.mean(abs_dy[i+1:i+1+ma_samps])
        print(i, ma_gradient_l, ma_gradient_r, ma_gradient_l / ma_gradient_r)
        if (ma_gradient_l / ma_gradient_r) < 0.05:
            qrs_lim_0 = i
            break
    # Didn't manage to find limit
    else:
        print('dah')
        qrs_lim_0 = qrs_ind - h_max_qrs_width
    
    print('\n\n Moving right \n\n')
    # Right
    ma_gradient_prev = np.mean(abs_dy[qrs_ind-h_max_qrs_width:qrs_ind+h_max_qrs_width])
    for i in range(qrs_ind, qrs_ind+h_max_qrs_width):
        ma_gradient_current = np.mean(abs_dy[i:i+ma_samps])
        print(ma_gradient_current, ma_gradient_prev, ma_gradient_current / ma_gradient_prev)
        if ma_gradient_current < ma_gradient_prev * .15:
            qrs_lim_1 = i
            break
        ma_gradient_prev = ma_gradient_current
    # Didn't manage to find limit
    else:
        print('doh')
        qrs_lim_1 = qrs_ind + h_max_qrs_width

    # Assume the 
    
    
    # Figure out whether the qrs complex points up or down
    
    return qrs_lim_0, qrs_lim_1

In [172]:
qrs_lim_0, qrs_lim_1 = segment_beat(sig, center)

72 36
87 56.61387 55.81943 1.0142323201795505
86 58.3735825 54.4560075 1.0719401803373116
85 57.62708999999999 51.534807500000014 1.1182168479042047
84 53.90896 48.21555000000001 1.1180824443566442
83 47.39116 45.42284000000001 1.0433332658195742
82 38.844212500000005 43.6967525 0.8889496421959504
81 29.4690925 43.25648 0.6812642290819779
80 20.374029999999998 44.035890000000016 0.462668745879781
79 12.635372499999999 46.12505000000001 0.27393731822512923
78 6.923319999999999 49.4120075 0.14011412104638732
77 3.340769999999999 53.25471000000001 0.06273191610657533
76 1.4778074999999986 56.61387 0.02610327645857806


 Moving right 


54.4560075 16.272821875 3.346439106769857
55.81943 54.4560075 1.0250371366281306
54.5743725 55.81943 0.9776949083858435
50.2500725 54.5743725 0.9207631750598689
43.19426 50.2500725 0.8595860234828516
34.54647 43.19426 0.7997930743575651
25.552197500000005 34.54647 0.7396471332671618
17.294677500000002 25.552197500000005 0.6768371878778723
10.574677499999996

In [178]:
# Plot the lims
print(qrs_lim_0, qrs_lim_1)
plt.figure()
plt.plot(sig)
plt.plot(amplitude_envelope, 'k')
plt.plot(qrs_lim_0, sig[qrs_lim_0], 'r*')
plt.plot(qrs_lim_1, sig[qrs_lim_1], 'r*')
plt.plot(dy)
plt.plot(abs_dy)
plt.show()

76 123


99

87

In [19]:
fs = 360
max_qrs_duration = 0.2
max_qrs_len = int(max_qrs_duration * fs)
dy = np.diff(sig)
abs_dy = np.abs(dy)

In [170]:
plt.figure()
plt.plot(sig)
plt.plot(sig_smooth, 'k')
plt.plot(center, sig[center], 'r*')

plt.plot(dy)
plt.plot(abs_dy)
plt.show()

In [16]:
dy = np.diff(sig)

In [153]:
x = np.array([1,2,3]) ** 2

In [154]:
x

array([1, 4, 9])